In [36]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import tensorflow as tf
import edward as ed
from edward.models import Normal

import matplotlib.pyplot as plt
import seaborn as sns

from jupyterthemes import jtplot
jtplot.style('oceans16')
%matplotlib inline

In [6]:
df = pd.read_csv("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/fake_or_real_news.csv")

In [114]:
bb = df['text'][0]

In [37]:
wrds = word_tokenize(bb)
stopwords = nltk.corpus.stopwords.words('english')

In [39]:
final = [word for word in wrds if word.lower() not in stopwords]

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

In [56]:
vectorizer = CountVectorizer(min_df = 0)

In [57]:
class LCVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LCVectorizer, self).build_analyzer()
        return lambda doc: (lemmatizer.lemmatize(w) for w in analyzer(doc))

In [59]:
tf_vectorizer = LCVectorizer(max_df=0.95, min_df=2, stop_words='english', decode_error='ignore')

In [72]:
tf = tf_vectorizer.fit_transform(wrds)

In [64]:
from sklearn.decomposition import LatentDirichletAllocation

In [82]:
lda = LatentDirichletAllocation(n_components=11, max_iter=10,
                               learning_method='online',
                               learning_offset =50.,
                               random_state = 0)

In [83]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=11, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [84]:
tf_features = tf_vectorizer.get_feature_names()

In [85]:
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [88]:
print_top_words(lda, tf_features, 40)


Topic #0:clinton afraid awkwardly smoke belief warned know cnn accusing people comey scandal cover lie credibility want obama really political just public associate hatch medium time campaign victory question stretch kgb gone fear assault hoover new act trump explanation reason like

Topic #1:fear setup lie like new fighting decided try nominee strategy foundation accusing reason credibility really tell director assault afraid kgb year awkwardly know doj campaign old violating countless medium republican hatch knew james war associate explanation conspiracy hoover presidential hillary

Topic #2:fbi people old right wing presidential hatch political james leadership act cnn bigger tell know going setup near obama bizarre panicked explanation public unprecedented clintonworld house medium fear victory scandal associate republican year just final democrat agent really new hubris

Topic #3:campaign time going want reason presidential final republican doj conspiracy act attack decided biza

In [87]:
first_topic = lda.components_[0]

In [91]:
from nltk.sentiment import sentiment_analyzer

(11, 89)